In [1]:
!pip install requests beautifulsoup4 selenium webdriver-manager pandas

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.0 MB 2.3 MB/s eta 0:00:05
    --------------------------------------- 0.2/10.0 MB 2.1 MB/s eta 0:00:05
   - -------------------------------------- 0.3/10.0 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.0 MB 2.3 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/10.0 MB 2.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/10.0 MB 2.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/10.0 MB 2.3 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/10.0 MB 2.4 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/10.0 MB 2.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.1/10.0 MB 2.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/10.0 MB 2.4 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/10.0 MB 2.4 MB/s eta 0:00:04
   ---

Docket numbers retrieved manually from the list maintained by findlaw.com. Only cases decided in 2018 and onward have their documents hosted on the Scotus website. Briefs from 2012 term onward are stored on scotusblog.com, although some of them are in a very messy format (i.e. a picture of a scanned document). Prior briefs used to be hosted by the ABA but they took them down.

In [1]:
docket_nos_path = "./docket_nos_valid.txt"

In [2]:
import pandas as pd
import re

df = pd.DataFrame(columns=['docket_number', 'year', 'url_list'])

current_year = None
rows = [] 

with open(docket_nos_path, 'r') as file:
    for line in file:
        line = line.strip()

        # Check for year-only lines. The year indicates when the decision was released.
        year_match = re.match(r'^<year>\s+(\d{4})$', line)
        if year_match:
            current_year = year_match.group(1)
            continue

        # Extract the docket number from regular lines
        match = re.search(r'No\.\s+([\w-]+)', line)
        if match:
            docket_number = match.group(1)
            # Use the current year for this entry and add to the rows list
            rows.append({'docket_number': docket_number, 'year': current_year})

df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)

print(df.head())


  docket_number  year url_list
0         17-71  2018      NaN
1       17-1676  2018      NaN
2       18-5181  2018      NaN
3        17-587  2018      NaN
4       17-7894  2018      NaN


In [3]:
print(len(df))

412


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re

# Setup WebDriver with headless option
options = Options()
options.add_argument("--headless")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to process a single docket page
def process_docket_page(docket_number):
    url = f"https://www.supremecourt.gov//docket/docketfiles/html/public/{docket_number}.html"
    pdf_urls = []
    try:
        driver.get(url)
    except Exception as e:
        print(f"Error loading page {url}: {str(e)}")
        return pdf_urls

    try:
        # Find all table rows
        rows = driver.find_elements(By.TAG_NAME, 'tr')
    except Exception as e:
        print(f"Error finding table rows: {str(e)}")
        return pdf_urls

    for row in rows:
        try:
            # Check if the row contains the text indicating a brief or document type
            # Might need to modify this to exclude cases where Motion is mentioned
            # Modify this to exclude cases where "Supplemental letter" is in text
            pattern = re.compile(r'^(?!.*\b(motion|supplemental letter)\b).*\b(brief of|reply of|brief amicus curiae)\b', re.IGNORECASE)
            
            if pattern.search(row.text):
                # Find all 'a' elements in this row
                # print(f"Match found in row: {row.text}")
                # Find the next sibling 'td' that contains the link
                next_td = row.find_element(By.XPATH, "./following-sibling::tr[1]/td[2]")
                # print(f"The next_td is {next_td.get_attribute('outerHTML')}")
                links = next_td.find_elements(By.TAG_NAME, 'a')
                # print(f"The links are: {links}")
                if len(links) > 0:
                    for link in links:
                        # Check if the link text is "Main Document"
                        # print(f"the link is {link.text}")
                        if link.text.strip().lower() == 'main document':
                            pdf_urls.append(link.get_attribute('href'))
        except Exception as e:
            print(f"Error processing row: {str(e)}")
            continue

    return pdf_urls


In [5]:
import time
import random

# List of docket numbers to process
docket_numbers = df['docket_number'].tolist()

# Dictionary to hold docket numbers and their associated PDF URLs
docket_pdf_urls = {}

# Try to do this directly with the dataframe rows, otherwise use a list.
for idx, row in df.iterrows():
    docket_number = row['docket_number']
    pdf_urls = process_docket_page(docket_number)
    docket_pdf_urls[docket_number] = pdf_urls
    df.at[idx, 'url_list'] = pdf_urls
    
    time.sleep(random.uniform(1, 2)) # Hopefully avoid rate limits

driver.quit()

In [6]:
print(len(docket_pdf_urls))

410


In [7]:
import requests

# Ensure you have the correct headers for the request if needed
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
}

num_briefs = 0

for docket_number, urls_list in docket_pdf_urls.items():
    num = 1
    for url in urls_list:
        response = requests.get(url, headers=headers, stream=True)
        if response.status_code == 200:
            with open(f'./brief_pdfs/Docket{docket_number}_Brief{num:03}.pdf', 'wb') as f:
                f.write(response.content)
            
            num += 1
            num_briefs += 1
            # print("pdf saved successfully, I sleep now")
            time.sleep(random.uniform(1, 2))
        else:
            print(f"Failed to download PDF for docket number {docket_number}. Status code: {response.status_code}")


In [8]:
print(f"You scraped {num_briefs} briefs.")

You scraped 4377 briefs.


In [1]:
df.to_json('./scraped_briefs.json', orient='records')

NameError: name 'df' is not defined

This is for DOJ's archive of OSG briefs, which would serve as a backup. Probably better download directly from supremecourt.gov though.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

# Setup Chrome options for headless mode
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

# Initialize WebDriver
driver = webdriver.Chrome(options=options)

# Function to download PDFs for a given type
def download_pdfs_for_type(type_value):
    driver.get('https://www.justice.gov/osg/supreme-court-briefs')

    # Select the type from the dropdown
    select = Select(driver.find_element_by_id('edit-type'))
    select.select_by_value(type_value)

    # Click the search button
    driver.find_element_by_id('edit-submit-brief-views').click()

    # Wait for the page to load
    time.sleep(5)  # Adjust this delay as necessary

    # Use BeautifulSoup to parse driver.page_source, find PDF links, and download them
    # Navigate pages if necessary and repeat the download process

    # Make sure to handle exceptions and edge cases

# List of types to process
types = ['merits_stage_brief', 'merits_stage_reply_brief', 'merits_stage_amicus_brief']

for type_value in types:
    download_pdfs_for_type(type_value)

driver.quit()
